## Setup

In [ ]:
%pip install -qU writer-sdk

### Initialization

Sign up for [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) and follow this [Quickstart](https://dev.writer.com/api-guides/quickstart) to obtain an API key. Then, set the WRITER_API_KEY environment variable:

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()

### Usage

#### Inputs definition

In [ ]:
inputs=[
    {
        "id": "ID (name) of input field",
        "value": [
            "Value of input field"
        ]
    }
] #each app have certain inputs, this example should be overridden according to app, it's it you've specified

#### Invocation

In [ ]:
#you should create an app at Writer AI studio before usage

if not os.getenv("APPLICATION_ID"): #gneration is available for all app types, but streaming only for "Research assistant"
    os.environ["APPLICATION_ID"] = getpass.getpass("Enter ID of Application you want to use: ")

application_generate_content_response = client.applications.generate_content(
    application_id=os.getenv("APPLICATION_ID"),
    inputs=inputs
)

application_generate_content_response.suggestion

#### Streaming

In [ ]:
application_generate_content_response = client.applications.generate_content(
    application_id=os.getenv("APPLICATION_ID"),
    inputs=inputs,
    stream=True #only for research assistant
)

for chunk in application_generate_content_response:
    delta = chunk.delta
    if delta.title:
        print(delta.title)
    if delta.stages:
        print(delta.stages[0].content)
    if delta.content:
        print(delta.content, end="", flush=True)

#### Retrieving app info

In [ ]:
application = client.applications.retrieve(
    application_id=os.getenv("APPLICATION_ID"),
)

application.to_json()

#### Retrieving list of apps

In [ ]:
page = client.applications.list()

for app in page.data:
    print(app.name + " " + app.id)

#### Retrieving list of apps with pagination params

In [ ]:
page_with_params = client.applications.list(order="asc", after=os.getenv("APPLICATION_ID"), limit=2)

for app in page_with_params.data:
    print(app.name + " " + app.id)